# 1 Retrieve all tables which are defined in writers


## 1.1 Example on how to filter range of writers

In [3]:
WRITERS = ('Power_BI', 'BIPOW')
START_DATE='2022-01-01'

In [4]:
'Power_BI.xx'.startswith(WRITERS)

True

## 1.2 Custom functions and imports

In [42]:
import requests
import pandas as pd
import networkx as nx

In [6]:
TOKEN = 'XXX' # expires in 12 hours

def list_configs(is_deleted="false"):
    
    url = f'https://connection.eu-central-1.keboola.com/v2/storage/components/keboola.wr-db-snowflake/configs?isDeleted={is_deleted}'
    headers = {}
    headers["X-StorageApi-Token"] = TOKEN
    res = requests.get(url, headers=headers)
    assert res.status_code==200, print(res.json())
    return res.json()

def getIOurl(tabid):
    """
    The function maps corresponding urls to tables. 
    If the source/destination is not a table, it retains
    original IO
    """
    
    url = None
    try:
        if tabid.startswith("https"):
            url = tabid
        else:
            ts = tabid.split(".")
            url = f'https://connection.eu-central-1.keboola.com/admin/projects/390/storage/{ts[0]}.{ts[1]}/{ts[2]}'
    except AttributeError:
        pass
    return url

In [7]:
def get_reachable_nodes(graph, node):
    bfs = nx.bfs_tree(graph, source=node)
    return set(bfs.nodes)

def calculate_n_reachable_nodes(graph, node):
    """
    this is a directed graph. number of upstream nodes is the same.
    number of downstream nodes should differ.
    """
    return len(set(list(nx.ancestors(graph, node))))

## 1.3 Collecting all tables from writers
The tables are extracted from writers and keyed by urls to easily explore in keboola

In [11]:
# get all nondeleted writer configs
configs = list_configs()

# get all table names from writer configs
tables_in_wr = []
for c in configs:
    wrname = c['name']
    for row in c['rows']:
        if not row["isDisabled"]:
            tab_id = row['configuration']['parameters']['tableId']
            destination = row['configuration']['parameters']['dbName']
            tables_in_wr.append({'wr':wrname, 'tableId':tab_id, 'dbname':destination, 'url':getIOurl(tab_id)})
print(f"There are {len(configs)} writers with {len(tables_in_wr)} tables in total.")

There are 11 writers with 653 tables in total.


# 2 Load tables from GSheet
The following tables are required for PowerBI reports to work. Let's see first two rows.  
Then add writer name as additional column.

<span style="color:red"> NOTE: If writers are defined in google sheet, disable this. Alternatively add mappings if needed!</span>


In [12]:
tables_in_use_df = pd.read_csv("in/tables/tables_in_use.csv")
tables_in_use_df.head(2)

,Workspace,Report,Tables,note
0,ALL,BI_fronta,bi-fronta-list-1,table
1,ALL,BI_fronta,Priorita,table


In [13]:
# custom adding of writer column into the dataframe 
tables_in_use_df["wr"] = 'Power_BI'
tables_in_use_df.head(2)

,Workspace,Report,Tables,note,wr
0,ALL,BI_fronta,bi-fronta-list-1,table,Power_BI
1,ALL,BI_fronta,Priorita,table,Power_BI


### 2.1 Upper for tables
* dropping duplicates
* converting to upper case

In [14]:
# tables from GSheet
tables_in_use2_df = tables_in_use_df.loc[tables_in_use_df.note=='table', ["Tables", "wr"]].drop_duplicates()
tables_in_use2_df["Tables"] = tables_in_use2_df.Tables.str.upper()
tables_in_use2_df["wr"] = tables_in_use2_df.wr.str.upper()
print(f"The dataframe from GSheet has {tables_in_use2_df.shape[0]} rows and {tables_in_use2_df.shape[1]} columns.")
tables_in_use2_df.head(2)

The dataframe from GSheet has 723 rows and 2 columns.


,Tables,wr
0,BI-FRONTA-LIST-1,POWER_BI
1,PRIORITA,POWER_BI


In [17]:
# Tables from writers
tables_in_wr_df = pd.DataFrame(tables_in_wr)
tables_in_wr_df["dbname"] = tables_in_wr_df.dbname.str.upper()
tables_in_wr_df["wr"] = tables_in_wr_df.wr.str.upper()
print(f"The dataframe from writers has {tables_in_wr_df.shape[0]} rows and {tables_in_wr_df.shape[1]} columns.")
tables_in_wr_df.head(2)

The dataframe from writers has 653 rows and 4 columns.


,wr,tableId,dbname,url
0,POWER_BI,out.c-00_PROD.BV_objednavka_hlavicka,BV_OBJEDNAVKA_HLAVICKA,https://connection.eu-central-1.keboola.com/ad...
1,POWER_BI,out.c-00_PROD.BV_objednavka_polozka,BV_OBJEDNAVKA_POLOZKA,https://connection.eu-central-1.keboola.com/ad...


### 2.2 Pairing existing tables from GSheet and from writers 
 * unused_tables --> tables which are not useful any more. All processes only leading to these can be turned off
 * used_tables --> important tables which need to be preserved

In [19]:
df_existing = pd.merge(left=tables_in_use2_df, right=tables_in_wr_df, left_on=["Tables", "wr"], right_on=["dbname", "wr"])
df_existing.drop_duplicates(inplace=True)
print(f"The dataframe which merges writers and GSheet has {df_existing.shape[0]} rows and {df_existing.shape[1]} columns.")

The dataframe which merges writers and GSheet has 244 rows and 5 columns.


In [20]:
unused_tables = [url for url in tables_in_wr_df.url.values if url not in df_existing.url.values]
used_tables = [url for url in tables_in_wr_df.url.values if url in df_existing.url.values]
print(f"There are {len(set(used_tables))} tables which are still used for PowerBI reports.")
print(f"There are {len(set(unused_tables))} redundant tables.")

There are 244 tables which are still used for PowerBI reports.
There are 353 redundant tables.


# 3 Construct the graph
Three graphs are about to be constructed:
* G --> the graph which describes all tables and processes in the project
* GG --> the same graph, but reversed
* GGS2 --> subset of the previous graph which includes all the nodes which are accessible from tables that were identified as redundant.

In all graphs, nodes are identified by keboola links to enable easy pairing and checking in the project

### 3.1 Load tables from lineage
The underlying data were obtained from Keboola transformation https://connection.eu-central-1.keboola.com/admin/projects/390/transformations-v2/keboola.snowflake-transformation/504414990 . The table storage_inputs_and_outputs.csv tracks every object and its impact on storage

In [21]:
df = pd.read_csv('in/tables/storage_inputs_and_outputs.csv')
df.head(2)

,id,region,project_id,project_name,name,component_id,component_name,component_type,STORAGE_INPUTS_SOURCES,STORAGE_INPUTS_DESTINATIONS,STORAGE_OUTPUTS_SOURCES,STORAGE_OUTPUTS_DESTINATIONS,bucket_id,isDisabled
0,505675972,eu-central-1.keboola.com,390,EMG reporting,lineage,keboola.sandboxes,Workspace,other,in.c-kds-team-ex-kbc-project-metadata-v2-50441...,NaN,NaN,NaN,NaN,False
1,505675972,eu-central-1.keboola.com,390,EMG reporting,lineage,keboola.sandboxes,Workspace,other,in.c-kds-team-ex-kbc-project-metadata-v2-50441...,NaN,NaN,NaN,NaN,False


### 3.2 Clean and enrich lineage data
Here some cleaning goes on. Also, a URL column must be added.
Next, it was identified that some data coming from telemetry are really useful. 

Therefore the lineage data are paired with the following:
* kbc_job.csv - information about jobs in Keboola
* kbc_component_configuration.csv - information about configurations of Keboola components
* kbc_component_configuration_version.csv - information about versions of the configurations above

In [22]:
def splitter(x):
    """
    Return the whole string or table name part
    """
    
    split_x = x
    try:
        split_x = str(x).split('.')[-1]
    except KeyError:
        split_x = x
    return split_x

def construct_url(row):
    url = 'NA'
    base_url = 'https://connection.' + row["region"] + '/admin/projects/' + str(row["project_id"]) + '/' 
    if row["component_type"]=='transformation':
        url = base_url + f'transformations-v2/{row["component_id"]}/{row["id"]}' 
    elif row["component_type"]=='other':
        url = base_url + f'workspaces/{row["id"]}' 
    elif row["component_type"]=='legacy transformation':
        url = base_url + f'transformations/bucket/{row["bucket_id"]}/transformation/{row["id"]}'
    else:
        url = base_url + f'components/{row["component_id"]}/{row["id"]}' 
    return url 

def construct_url_tele():
    url = 'NA'
    base_url = 'https://connection.' + row["region"] + '/admin/projects/' + str(row["project_id"]) + '/' 
    if row["component_type"]=='transformation':
        url = base_url + f'transformations-v2/{row["component_id"]}/{row["id"]}' 
    elif row["component_type"]=='other':
        url = base_url + f'workspaces/{row["id"]}' 
    elif row["component_type"]=='legacy transformation':
        url = base_url + f'transformations/bucket/{row["bucket_id"]}/transformation/{row["id"]}'
    else:
        url = base_url + f'components/{row["component_id"]}/{row["id"]}' 
    return url 


In [23]:
# clean data from lineage and add URL column
replace_values = {'\"' : '', '\n' : '', '[' : '', ']' : ''  }        

df.replace({"STORAGE_INPUTS_SOURCES": replace_values}, inplace=True)   
df.replace({"STORAGE_OUTPUTS_DESTINATIONS": replace_values}, inplace=True)   
# create a new dataframe with the columns we want

df["STORAGE_INPUTS_SOURCES_NAMES"]=df.STORAGE_INPUTS_SOURCES.apply(lambda x: splitter(x))
df["STORAGE_OUTPUTS_DESTINATIONS_NAMES"]=df.STORAGE_OUTPUTS_DESTINATIONS.apply(lambda x: splitter(x))

df = df.astype({'bucket_id':'Int64'}, errors='ignore')
df = df.astype({'id':'Int64'}, errors='ignore')

df["URL"] = df.apply(lambda x: construct_url(x), axis=1)

df["STORAGE_INPUTS_SOURCES_URL"]=df.STORAGE_INPUTS_SOURCES.apply(lambda x: getIOurl(x))
df["STORAGE_OUTPUTS_DESTINATIONS_URL"]=df.STORAGE_OUTPUTS_DESTINATIONS.apply(lambda x: getIOurl(x))

### 3.3 Create nodes and edges from lineage data

In [24]:
# url, table name pairs for source and destination tables
storage_inputs_sources = list(df.loc[~pd.isna(df.STORAGE_INPUTS_SOURCES_URL), ["URL", "STORAGE_INPUTS_SOURCES_URL"]].to_records(index=False))
storage_inputs_sources = [tuple(x) for x in (storage_inputs_sources)]
storage_outputs_destinations = list(df.loc[~pd.isna(df.STORAGE_OUTPUTS_DESTINATIONS_URL), ["URL", "STORAGE_OUTPUTS_DESTINATIONS_URL"]].to_records(index=False))
storage_outputs_destinations = [tuple(x) for x in (storage_outputs_destinations)]
# flip sources
storage_inputs_sources_flipped = [(x[1], x[0]) for x in storage_inputs_sources]
# flipping source table...
edges = (storage_inputs_sources_flipped + storage_outputs_destinations)

### 3.4 Calculate usage stats from KBC telemetry
a) data from jobs and configurations are merged    
b) calculate last job and total price  
c) merge with lineage data to add url column

In [25]:
#a)
# load jobs
jobsdf = pd.read_csv("in/tables/kbc_job.csv", parse_dates=["dst_timestamp"])
# load configs 
configs_df = pd.read_csv("in/tables/kbc_component_configuration.csv", parse_dates=["dst_timestamp"])
configs_df = configs_df.loc[~configs_df.kbc_configuration_is_deleted]
versions_df = pd.read_csv("in/tables/kbc_component_configuration_version.csv", parse_dates=["dst_timestamp", "configuration_updated_at"])
versions_df.head(2)

,kbc_component_configuration_id,kbc_branch_id,configuration_updated_at,change_description,configuration_version,last_version,token_id,token_name,dst_proj_single,dst_stack_single,dst_proj_company,dst_stack_company,dst_timestamp
0,390_kbc-eu-central-1_transformation_278194054,123_kbc-eu-central-1,2023-02-06 21:02:41,Transformation import_per_15min disabled,223,False,213848,papousek.radan@euromedia.cz,390,connection.eu-central-1.keboola.com,NaN,NaN,2023-02-07 00:09:59
1,390_kbc-eu-central-1_transformation_341173431,123_kbc-eu-central-1,2023-02-06 11:55:33,Change Scripts in Cenotvorba,91,True,213848,papousek.radan@euromedia.cz,390,connection.eu-central-1.keboola.com,NaN,NaN,2023-02-06 14:00:52


In [29]:
# getting the latest versions only
versions_df_latest= versions_df.sort_values('configuration_updated_at').groupby('kbc_component_configuration_id').tail(1)
versions_df_latest.head(2)

,kbc_component_configuration_id,kbc_branch_id,configuration_updated_at,change_description,configuration_version,last_version,token_id,token_name,dst_proj_single,dst_stack_single,dst_proj_company,dst_stack_company,dst_timestamp
28794,390_kbc-eu-central-1_keboola.wr-db-snowflake_2...,123_kbc-eu-central-1,2019-02-05 14:58:41,Configuration deleted,4,True,4162,jakub.turner@keboola.com,390,connection.eu-central-1.keboola.com,NaN,NaN,2022-08-10 16:04:31
18185,390_kbc-eu-central-1_keboola.ex-db-mssql_28435431,123_kbc-eu-central-1,2019-02-22 08:56:01,Configuration deleted,2,True,4164,david.zelenka@bootiq.io,390,connection.eu-central-1.keboola.com,NaN,NaN,2022-08-10 16:04:31


In [28]:
configs_df = pd.merge(configs_df, versions_df_latest.loc[:, ["kbc_component_configuration_id", "token_name", "configuration_updated_at"]])
jobsdf.head(2)

,kbc_job_id,kbc_component_configuration_id,kbc_branch_id,kbc_component_id,transformation_type,job_run_id,job_start_at,job_created_at,job_status,error_type,...,ds_backend_size,dwh_small_ratio,dwh_medium_ratio,dwh_large_ratio,backend_size,dst_proj_single,dst_stack_single,dst_proj_company,dst_stack_company,dst_timestamp
0,525058892_kbc-eu-central-1,390_kbc-eu-central-1_keboola.snowflake-transfo...,123_kbc-eu-central-1,keboola.snowflake-transformation,SQL,525056190.525058893,2023-01-27 15:56:20.000,2023-01-27 15:56:20.000,success,NaN,...,NaN,0.96837,0.0,0.0,multi,390,connection.eu-central-1.keboola.com,NaN,NaN,2023-01-27 17:59:46
1,524898401_kbc-eu-central-1,390_kbc-eu-central-1_keboola.wr-db-snowflake_1...,123_kbc-eu-central-1,keboola.wr-db-snowflake,None,524752412.524898402,2023-01-27 06:40:54.000,2023-01-27 06:24:49.000,success,NaN,...,NaN,NaN,NaN,NaN,NaN,390,connection.eu-central-1.keboola.com,NaN,NaN,2023-01-27 09:59:57


In [30]:
config_stats_df = pd.merge(left=configs_df, right=jobsdf, on="kbc_component_configuration_id", validate="one_to_many")
group_cols = ["kbc_component_configuration_id", "kbc_component_id_x", "kbc_component_configuration", "kbc_component", "configuration_id_num", "kbc_component_type", "token_name_x", "configuration_updated_at"]
config_stats_df = config_stats_df.loc[config_stats_df.dst_timestamp_y>START_DATE].groupby(group_cols).agg(total_consumption=("job_billed_credits_used", "sum"), 
                                                     last_usage=("dst_timestamp_y", "last"))
config_stats_df.reset_index(inplace=True)
# some numeric ids are not exactly numeric. the data is converted into numeric replacing the rest with NAs. Then to integer by using pandas native Int64 type
config_stats_df["configuration_id_num"]=pd.to_numeric(config_stats_df["configuration_id_num"], errors="coerce").astype("Int64")
print(f"Shape of merged table {config_stats_df.shape}")
config_stats_df.head(2)

Shape of merged table (439, 10)


,kbc_component_configuration_id,kbc_component_id_x,kbc_component_configuration,kbc_component,configuration_id_num,kbc_component_type,token_name_x,configuration_updated_at,total_consumption,last_usage
0,390_kbc-eu-central-1_blueskydigital.wr-ftp_334...,blueskydigital.wr-ftp_kbc-eu-central-1,Samba,FTP/FTPS,334091535,writer,par@euromedia.cz,2021-10-20 16:29:39,30.946479,2022-12-08 13:25:17
1,390_kbc-eu-central-1_bootiq.ex-mssql-extended_...,bootiq.ex-mssql-extended_kbc-eu-central-1,HEG_Neoluxor,MSSQL-extended,122782245,extractor,michal.kadera@bootiq.io,2022-03-23 01:25:03,0.000000,2022-12-08 13:25:17


**right join is used below to leave room for transformation buckets**

In [31]:
# c) merging lineage and telemetry data to add urls to telemetry stats
config_stats_df_final = pd.merge(left=df.loc[:, ["id", "component_id", "name", "component_type", "URL", "component_name"]].drop_duplicates(["id", "component_id"]), 
         right=config_stats_df, 
         left_on=["name", "id"], 
         right_on=["kbc_component_configuration", "configuration_id_num"]
         , validate="one_to_one"
        , how="right"
        )
config_stats_df_final.set_index("URL", inplace=True)

In [32]:
config_stats_df_final.head(2)

,id,component_id,name,component_type,component_name,kbc_component_configuration_id,kbc_component_id_x,kbc_component_configuration,kbc_component,configuration_id_num,kbc_component_type,token_name_x,configuration_updated_at,total_consumption,last_usage
URL,,,,,,,,,,,,,,,
https://connection.eu-central-1.keboola.com/admin/projects/390/components/blueskydigital.wr-ftp/334091535,334091535,blueskydigital.wr-ftp,Samba,writer,FTP/FTPS,390_kbc-eu-central-1_blueskydigital.wr-ftp_334...,blueskydigital.wr-ftp_kbc-eu-central-1,Samba,FTP/FTPS,334091535,writer,par@euromedia.cz,2021-10-20 16:29:39,30.946479,2022-12-08 13:25:17
NaN,<NA>,NaN,NaN,NaN,NaN,390_kbc-eu-central-1_bootiq.ex-mssql-extended_...,bootiq.ex-mssql-extended_kbc-eu-central-1,HEG_Neoluxor,MSSQL-extended,122782245,extractor,michal.kadera@bootiq.io,2022-03-23 01:25:03,0.000000,2022-12-08 13:25:17


In [33]:
recs = df.loc[:, ["URL", "name", "component_id", "component_type", "bucket_id", "isDisabled"]].to_records(index=False).tolist()
len(set(recs))

753

In [34]:
recs[:3]

[('https://connection.eu-central-1.keboola.com/admin/projects/390/workspaces/505675972',
  'lineage',
  'keboola.sandboxes',
  'other',
  <NA>,
  False),
 ('https://connection.eu-central-1.keboola.com/admin/projects/390/workspaces/505675972',
  'lineage',
  'keboola.sandboxes',
  'other',
  <NA>,
  False),
 ('https://connection.eu-central-1.keboola.com/admin/projects/390/workspaces/477288885',
  'REVOLT_10_HEG_NLX',
  'keboola.sandboxes',
  'other',
  <NA>,
  False)]

NOTE: for legacy transformations, it is not possible to break consumption into individual transformation. Rather, the whole consumption is always allocated to a transformation bucket. Therefore, in the graph, there will be some double counting. 

## 3.5 Setting up data structures which contain all underlying data needed to construct the graph
#### 3.5.1 Nodes for Keboola components

In [37]:
# setting up graph nodes
nodes_components=[]
for r in set(recs):
    
    url = r[0]
    comment = ""
    
    last_usage = 'NA'
    total_consumption = 0
    
    try:
        last_usage=config_stats_df_final.loc[url, "last_usage"]
        last_update=config_stats_df_final.loc[url, "configuration_updated_at"]
        last_updater=config_stats_df_final.loc[url, "token_name_x"]
        total_consumption=config_stats_df_final.loc[url, "total_consumption"]
    except KeyError:
        #print(r)
        # check whether it is a legacy transformation
        if r[3] == 'legacy transformation':
            # grab data for the whole bucket
            last_usage=config_stats_df_final.loc[((config_stats_df_final.kbc_component=="Legacy Transformations")
                                                 &(config_stats_df_final.configuration_id_num==r[4])), "last_usage"].values[0] 
            total_consumption=config_stats_df_final.loc[((config_stats_df_final.kbc_component=="Legacy Transformations")
                                                 &(config_stats_df_final.configuration_id_num==r[4])), "total_consumption"].values[0]

            last_update=config_stats_df_final.loc[((config_stats_df_final.kbc_component=="Legacy Transformations")
                                                 &(config_stats_df_final.configuration_id_num==r[4])), "configuration_updated_at"].values[0]
            last_updater=config_stats_df_final.loc[((config_stats_df_final.kbc_component=="Legacy Transformations")
                                                 &(config_stats_df_final.configuration_id_num==r[4])), "token_name_x"].values[0]

            
            comment = "BUCKET_VALUE!"
    t = (url, {'name':r[1], 
                "component_id":r[2], 
                "component_type":r[3], 
                "last_usage":last_usage,
                "total_consumption":total_consumption, 
                "comment":comment,
                "isDisabled":r[5], 
               "last_update":last_update,
               "last_updater":last_updater
               
               }
        )

    nodes_components.append(t)
    # IGNORE OTHER TYPES OF COMPONENTS... BUT OTHER KINDS OF COMPONENTS SHOULD BE HAVE URLS
print("Display of a few nodes:")
nodes_components[:3]

Display of a few nodes:


[('https://connection.eu-central-1.keboola.com/admin/projects/390/transformations/bucket/306522285/transformation/353857266',
  {'name': 'Naklady_marketing',
   'component_id': nan,
   'component_type': 'legacy transformation',
   'last_usage': numpy.datetime64('2022-05-18T12:50:15.000000000'),
   'total_consumption': 20.56477888812,
   'comment': 'BUCKET_VALUE!',
   'isDisabled': False,
   'last_update': numpy.datetime64('2022-06-24T14:21:20.000000000'),
   'last_updater': 'suchanek.vojtech@euromedia.cz'}),
 ('https://connection.eu-central-1.keboola.com/admin/projects/390/transformations-v2/keboola.snowflake-transformation/384903704',
  {'name': 's10HegEMGDoklPrvotni',
   'component_id': 'keboola.snowflake-transformation',
   'component_type': 'transformation',
   'last_usage': Timestamp('2022-12-08 13:25:17'),
   'total_consumption': 187.412933141,
   'comment': '',
   'isDisabled': False,
   'last_update': Timestamp('2022-06-28 11:05:35'),
   'last_updater': 'dvorak.jiri@euromedia.c

#### 3.5.2 Nodes for tables in Keboola storage
The tables do not have most of the attributes the components do

In [40]:
nodes_tables = []
for i, r in df.loc[~pd.isna(df.STORAGE_INPUTS_SOURCES, )].iterrows():
    if not r["STORAGE_INPUTS_SOURCES"].startswith("https"):
        ts = r["STORAGE_INPUTS_SOURCES"].split(".")
        nodes_tables.append((r["STORAGE_INPUTS_SOURCES_URL"], {'name':ts[-1], 'component_type':'table', 'table_id':r["STORAGE_INPUTS_SOURCES"]}))

for i, r in df.loc[~pd.isna(df.STORAGE_OUTPUTS_DESTINATIONS, )].iterrows():
    if not r["STORAGE_OUTPUTS_DESTINATIONS"].startswith("https"):
        ts = r["STORAGE_OUTPUTS_DESTINATIONS"].split(".")
        nodes_tables.append((r["STORAGE_OUTPUTS_DESTINATIONS_URL"], {'name':ts[-1], 'component_type':'table', 'table_id':r["STORAGE_OUTPUTS_DESTINATIONS"]}))

nodes_tables[:3]        

[('https://connection.eu-central-1.keboola.com/admin/projects/390/storage/in.c-kds-team-ex-kbc-project-metadata-v2-504415511/configurations',
  {'name': 'configurations',
   'component_type': 'table',
   'table_id': 'in.c-kds-team-ex-kbc-project-metadata-v2-504415511.configurations'}),
 ('https://connection.eu-central-1.keboola.com/admin/projects/390/storage/in.c-kds-team-ex-kbc-project-metadata-v2-504415511/transformations-inputs',
  {'name': 'transformations-inputs',
   'component_type': 'table',
   'table_id': 'in.c-kds-team-ex-kbc-project-metadata-v2-504415511.transformations-inputs'}),
 ('https://connection.eu-central-1.keboola.com/admin/projects/390/storage/in.c-NLX_Neoluxor/objekt-ucetnipohyb',
  {'name': 'objekt-ucetnipohyb',
   'component_type': 'table',
   'table_id': 'in.c-NLX_Neoluxor.objekt-ucetnipohyb'})]

### 3.6 Construct the graph
1. A table or a component can be a node
2. Edges are retrieved from the lineage transformation
3. Note here, that an edge can be defined as (A, B) for outputs, but (B, A) for inputs

In [43]:
node_list = nodes_components + nodes_tables
G = nx.DiGraph()
G.add_nodes_from(node_list)
G.add_edges_from(edges)

### 3.7 Revert a graph

In [45]:
GG = nx.reverse_view(G)

### 3.8 Subgraph
The descendants method provides all nodes accessible from a particular node. Therefore the subgraph contains all nodes that can be traversed beginning with one of the unused tables

In [48]:
nodes_subgraph = []
missing_nodes =[]
for tab in unused_tables:
    try:
        nodes_subgraph = nodes_subgraph + [tab] + list(nx.descendants(GG, tab))
    except nx.NetworkXError:
        missing_nodes.append(tab)
        
nodes_subgraph = set(nodes_subgraph)
GGS2 = nx.subgraph(GG, nodes_subgraph)

print(f"Subgraph has {len(GGS2.nodes)} nodes (vs {len(G.nodes)} for the whole graph).")
print(f"There is {len(set(missing_nodes))} unconnected nodes which cannot be backtracked.")

Subgraph has 1573 nodes (vs 2418 for the whole graph).
There is 22 unconnected nodes which cannot be backtracked.


<span style="color:blue">**This means that we do not have any processes which populate these tables. They can probably be deleted. They should definitely be exluded from all writers.**</span>

In [64]:
missing_nodes[:5]

['https://connection.eu-central-1.keboola.com/admin/projects/390/storage/out.c-radan/discount_spending',
 'https://connection.eu-central-1.keboola.com/admin/projects/390/storage/out.c-emg_danek/rfm_all_publisher',
 'https://connection.eu-central-1.keboola.com/admin/projects/390/storage/in.c-keboola-ex-google-drive-390144083/zakaznicky-servis-agenti-agenti',
 'https://connection.eu-central-1.keboola.com/admin/projects/390/storage/out.c-emg_simonovska/rfm_data_fin_temp',
 'https://connection.eu-central-1.keboola.com/admin/projects/390/storage/in.c-revolt-bi-app-daktela-345938057/emg_groups']

# 4 Graph Analysis
### 4.1 Investigate the importance of nodes
For each node, we compare number of ancestors. Ancestors are all the nodes which could be traversed going from the source (ie from writer). 
If the number of nodes which can be traversed is the same for the whole graph (GG) and for the subgraph (GGS2), it means, the node is only relevant in the subgraph context and can therefore be turned off

**EDIT: There was a bug before. Before the edit I was comparing number of incoming and outgoing edges for each node. This is not sufficient for comparison.**

In [49]:
todelete = []

for node in GGS2.nodes():
    n_reach_full = calculate_n_reachable_nodes(GG, node) #-- these are upstream nodes which have to be the same
    n_reach_sub = calculate_n_reachable_nodes(GGS2, node)
    
    if n_reach_full == n_reach_sub:
        todelete.append(node)
    
print(f"\nOut of {len(GGS2.nodes())} nodes in the graph, {len(todelete)} nodes can be safely deleted.")


Out of 1573 nodes in the graph, 378 nodes can be safely deleted.


In [50]:
todelete[:3]

['https://connection.eu-central-1.keboola.com/admin/projects/390/transformations/bucket/306522285/transformation/353857266',
 'https://connection.eu-central-1.keboola.com/admin/projects/390/transformations/bucket/101074842/transformation/106565461',
 'https://connection.eu-central-1.keboola.com/admin/projects/390/transformations/bucket/101074842/transformation/117166019']

### 4.2 Which unused elements have the lowest number of connections?
Here, we just check how many connections there are for redundant nodes

In [51]:
from collections import defaultdict

connection_counter = defaultdict(list)
for t in todelete:
    if t in missing_nodes:
        continue
    
    n_in_subgraph = GGS2.in_degree(t)
    n_out_subgraph = GGS2.out_degree(t)
    nconnections = n_in_subgraph + n_out_subgraph
    connection_counter[nconnections].append((t, G.nodes[t]))
    
print(sorted(connection_counter)  )  

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 18, 21, 24, 28, 30]


### 4.3 If we exclude tables and workspaces, how many connections there are?

In [54]:
# exlude tables and workspaces
from collections import defaultdict

connection_counter_trans = defaultdict(list)
for t in todelete:
    if t in missing_nodes:
        continue
    if G.nodes[t]['component_type'] not in ['table', 'Workspace']:
        n_in_subgraph = GGS2.in_degree(t)
        n_out_subgraph = GGS2.out_degree(t)
        nconnections = n_in_subgraph + n_out_subgraph
        connection_counter_trans[nconnections].append((t, G.nodes[t]))

In [53]:
sorted(connection_counter_trans)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 18, 21, 24, 28, 30]

In [57]:
connection_counter_trans[30]

[('https://connection.eu-central-1.keboola.com/admin/projects/390/workspaces/501143166',
  {'name': 'PanKostkaJenCashFlow',
   'component_id': 'keboola.sandboxes',
   'component_type': 'other',
   'last_usage': Timestamp('2023-02-02 10:03:21'),
   'total_consumption': 0.0,
   'comment': '',
   'isDisabled': False,
   'last_update': Timestamp('2023-01-12 11:44:07'),
   'last_updater': 'dvorak.jiri@euromedia.cz'})]

### 4.4 Collect processes to delete into a data frame

The dataframe can be sorted by last usage or total consumption to begin turning off orchestration with big impact first. 

**NOTE: Legacy transformations have "BUCKET_VALUE" in comment. This means that total consumption relates to the whole bucket, not individual transformations. However, an individual transformation can be switched off, we just cannot quantify its effect on consumption.**

In [58]:
pd.set_option('display.max_colwidth', None)
records=[]
for t in todelete:
    if t in missing_nodes:
        continue
    if G.nodes[t]['component_type'] not in ['table', 'Workspace']:
        n_out_connections = GGS2.in_degree(t)
        n_in_connections = GGS2.out_degree(t)
        node = G.nodes[t]
        node["url"]=t
        node["n_in_connections"] = n_in_connections
        node["n_out_connections"] = n_out_connections
        records.append(node)

In [59]:
delete_df = pd.DataFrame().from_records(records)
delete_df.sort_values(by="total_consumption", ascending=False, inplace=True)

In [60]:
delete_df.head(5)

,name,component_id,component_type,last_usage,total_consumption,comment,isDisabled,last_update,last_updater,url,n_in_connections,n_out_connections
47,NLX_Nakup,NaN,legacy transformation,2022-12-08 13:25:17,14344.144831,BUCKET_VALUE!,False,2023-01-11 09:32:45,papousek.radan@euromedia.cz,https://connection.eu-central-1.keboola.com/admin/projects/390/transformations/bucket/31076208/transformation/137780956,6,1
86,Saldo,NaN,legacy transformation,2022-12-08 13:25:17,14344.144831,BUCKET_VALUE!,False,2023-01-11 09:32:45,papousek.radan@euromedia.cz,https://connection.eu-central-1.keboola.com/admin/projects/390/transformations/bucket/31076208/transformation/93262189,3,2
30,Zasilky_NLX,NaN,legacy transformation,2022-12-08 13:25:17,14344.144831,BUCKET_VALUE!,False,2023-01-11 09:32:45,papousek.radan@euromedia.cz,https://connection.eu-central-1.keboola.com/admin/projects/390/transformations/bucket/31076208/transformation/124110545,6,1
99,nlx_zpozdeni,NaN,legacy transformation,2022-12-08 13:25:17,14344.144831,BUCKET_VALUE!,False,2023-01-11 09:32:45,papousek.radan@euromedia.cz,https://connection.eu-central-1.keboola.com/admin/projects/390/transformations/bucket/31076208/transformation/138301980,16,8
45,NLX_trzby,NaN,legacy transformation,2022-12-08 13:25:17,14344.144831,BUCKET_VALUE!,False,2023-01-11 09:32:45,papousek.radan@euromedia.cz,https://connection.eu-central-1.keboola.com/admin/projects/390/transformations/bucket/31076208/transformation/123126774,17,1


### 4.5 Tests on how to turn stuff off
#### 4.5.1 Most consuming component
Here, we take the component which is most often used first.

In [61]:
# get the transformation which consumes most
delete_df.sort_values(by="total_consumption", ascending=False, inplace=True)
TESTURL = delete_df.head(1).url.values[0]
print("url of the most consuming component:")
TESTURL

url of the most consuming component:


'https://connection.eu-central-1.keboola.com/admin/projects/390/transformations/bucket/31076208/transformation/137780956'

To assess whether the component can be turned off, we need to get all the components in the path from one of the tables in the writer to this component. This can be achieved again by using the ancestors method

In [63]:
# get all the predecessors from the subgraph
for node in nx.ancestors(GGS2, TESTURL):
    flagged_del = node in todelete
    print(f"The component {node} has been flagged to delete: {flagged_del}")

The component https://connection.eu-central-1.keboola.com/admin/projects/390/storage/out.c-nlx/NLX_polozky_nakup has been flagged to delete: True


#### 4.5.1 Random component
Here, the exercise is repeated, albeit for a component picked at random

In [64]:
# pick some transformation at random
import numpy as np
RAND = np.random.randint(delete_df.shape[0])
print(f"The random integer picked: {RAND}")

The random integer picked: 53


In [65]:
# get the transformation which consumes most
delete_df.sort_values(by="total_consumption", ascending=False, inplace=True)
TESTURL = delete_df.head(RAND).tail(1).url.values[0]
print("url of a randomly picked component:")
TESTURL

url of a randomly picked component:


'https://connection.eu-central-1.keboola.com/admin/projects/390/transformations/bucket/243731185/transformation/300528183'

In [66]:
# get all the predecessors from the subgraph
for node in nx.ancestors(GGS2, TESTURL):
    flagged_del = node in todelete
    print(f"The component {node} has been flagged to delete: {flagged_del}")

The component https://connection.eu-central-1.keboola.com/admin/projects/390/storage/out.c-luxor_2_0/sales_per_title_nlx_hybrid has been flagged to delete: True
The component https://connection.eu-central-1.keboola.com/admin/projects/390/transformations/bucket/243732112/transformation/298196203 has been flagged to delete: True
The component https://connection.eu-central-1.keboola.com/admin/projects/390/transformations/bucket/243732112/transformation/298181039 has been flagged to delete: True
The component https://connection.eu-central-1.keboola.com/admin/projects/390/transformations/bucket/243732112/transformation/298624980 has been flagged to delete: True
The component https://connection.eu-central-1.keboola.com/admin/projects/390/transformations/bucket/272963466/transformation/303811301 has been flagged to delete: True
The component https://connection.eu-central-1.keboola.com/admin/projects/390/storage/out.c-jdo9bdim-dev/o9POSDimCas has been flagged to delete: True
The component http